# **PyACE Cu-I pacemaker example**

## pip

In [1]:
import sys
!{sys.executable} -m pip install mpi4py pyace

In [3]:
!pip install -e /Users/mitch/github/python-ace-alphataubio

Obtaining file:///Users/mitch/github/python-ace-alphataubio
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pyace (pyproject.toml) ... done
  Created wheel for pyace: filename=pyace-0.2.7+180.g5c5d2d3-0.editable-cp312-cp312-macosx_11_0_arm64.whl size=33863 sha256=3bdb2e79f9ca6e9a31dad9ec67b284383a5e678ff7dc1756faaf9309f394162d
  Stored in directory: /private/var/folders/6t/_hgyp_2n3dx4sjtyq01p9ll00000gn/T/pip-ephem-wheel-cache-9vtejg70/wheels/b8/16/79/61a172e41f78891f5248dc9837655dfd16724c29ddfdb7844d
Successfully built pyace


## read data

In [5]:
import pandas as pd

df=pd.read_pickle("./Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip", compression="gzip")
print(df)

                                                name  \
0                A1:FHI-aims/PBE/tight:elastic:s_e_0   
1         A1:FHI-aims/PBE/tight:murnaghan:strain_1_0   
2    A1:FHI-aims/PBE/tight:phonon:supercell_phonon_0   
3        A1:FHI-aims/PBE/tight:murnaghan:strain_1_02   
4        A1:FHI-aims/PBE/tight:murnaghan:strain_0_98   
..                                               ...   
100     A2:FHI-aims/PBE/tight:elastic:s_01_e_0_05000   
101    A1:FHI-aims/PBE/tight:elastic:s_01_e_m0_05000   
102    A3:FHI-aims/PBE/tight:elastic:s_01_e_m0_05000   
103    A2:FHI-aims/PBE/tight:elastic:s_01_e_m0_05000   
104    A2:FHI-aims/PBE/tight:elastic:s_23_e_m0_05000   

                                             ase_atoms  energy_corrected  \
0    (Atom('Cu', [np.float64(0.0), np.float64(0.0),...         -3.699843   
1    (Atom('Cu', [np.float64(0.0), np.float64(0.0),...         -3.699841   
2    (Atom('Cu', [np.float64(4.3368086899420173e-19...       -236.789603   
3    (Atom('Cu', [np.fl

In [27]:
%load_ext autoreload
%autoreload 2
    
import sys

# pythonpath of fitsnap relative to this example directory
sys.path.append('../../..')

# change this to your lammps/python directory
sys.path.append('../../../../lammps-alphataubio/python')

from mpi4py import MPI
import numpy as np
from fitsnap3lib.fitsnap import FitSnap

settings = {
    "PYACE": {
        "elements": "Cu",
        "cutoff": 7.0,
        "delta_spline_bins": 0.001,
        "embeddings": {
            "ALL": {
                "npot": "FinnisSinclairShiftedScaled", 
                "fs_parameters": [1, 1, 1, 0.5], 
                "ndensity": 2
            }
        },
        "bonds": {
            "ALL": {
                "radbase": "ChebExpCos", 
                "radparameters": [5.25], 
                "rcut": 7.0, 
                "dcut": 0.01, 
                "NameOfCutoffFunction": "cos"
            }
        },
        "functions": {
            "UNARY": {
                "nradmax_by_orders": [15, 3, 2, 2, 1], 
                "lmax_by_orders": [0, 2, 2, 1, 1]
            }
        },
    },
    "CALCULATOR": { "calculator": "PyACE", "energy": 1, "force": 0, "stress": 0 },
    "ESHIFT": { "Cu": 0.0 },
    "SOLVER": { "solver": "RIDGE", "compute_testerrs": 1, "detailed_errors": 1 },
    "RIDGE": { "alpha": 1e-6 },
    "SCRAPER": { "scraper": "PACEMAKER" },
    "PATH": { "dataPath": "." },
    "OUTFILE": { "metrics": "Cu_I_metrics.md", "potential": "Cu_I" },
    "REFERENCE": {
        "units": "metal",
        "atom_style": "atomic",
        "pair_style": "zero 6.0",
        "pair_coeff": "* *"
    },
    "EXTRAS": { "dump_dataframe": 1 },
    "GROUPS": {
        "group_sections": "name training_size testing_size eweight fweight vweight",
        "group_types": "str float float float float float",
        "smartweights": 0,
        "random_sampling": 0,
        "Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip": "0.8    0.2    1.0    1.0    1.00E-08"
    }
}

fs = FitSnap(settings, comm=MPI.COMM_WORLD, arglist=["--overwrite"])
fs.scrape_configs()
print(len(fs.data))
print(fs.data[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PyACE config: elements=['Cu'], cutoff=7.0
Creating BBasisConfiguration using create_multispecies_basis_config
Config: elements=['Cu'], cutoff=7.0
Successfully created ACEBBasisSet using proper PyACE API
Successfully created pyace calculator with 82 basis functions

         Forcing testing size to add up properly.

Loading pacemaker file: ./Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip
Loaded 105 configurations from pacemaker files
'scrape_configs' took 17.65 ms on rank 0
105
{'File': 'Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip', 'Group': 'Cu_df1_A1_A2_A3_EV_elast_pho', 'Positions': array([[0., 0., 0.]]), 'AtomTypes': array([1]), 'NumAtoms': 1, 'QMLattice': array([[0.     , 1.81563, 1.81563],
       [1.81563, 0.     , 1.81563],
       [1.81563, 1.81563, 0.     ]]), 'Energy': -3.699842546498985, 'Forces': array([[0., 0., 0.]]), 'Stress': array([0., 0., 0., 0., 0., 0.]), 'Lattice': array([[ 2.56768857e+00,  1.

In [28]:
fs.process_configs()
print(fs.pt.shared_arrays['a'].array)

DEBUG: Processing config 0 with 1 atoms
DEBUG: Processing config 1 with 1 atoms
DEBUG: Processing config 2 with 64 atoms
DEBUG: Processing config 3 with 1 atoms
DEBUG: Processing config 4 with 1 atoms
DEBUG: Processing config 5 with 1 atoms
DEBUG: Processing config 6 with 1 atoms
DEBUG: Processing config 7 with 2 atoms
DEBUG: Processing config 8 with 2 atoms
DEBUG: Processing config 9 with 1 atoms
DEBUG: Processing config 10 with 96 atoms
DEBUG: Processing config 11 with 2 atoms
DEBUG: Processing config 12 with 2 atoms
DEBUG: Processing config 13 with 2 atoms
DEBUG: Processing config 14 with 1 atoms
DEBUG: Processing config 15 with 2 atoms
DEBUG: Processing config 16 with 2 atoms
DEBUG: Processing config 17 with 2 atoms
DEBUG: Processing config 18 with 1 atoms
DEBUG: Processing config 19 with 1 atoms
DEBUG: Processing config 20 with 2 atoms
DEBUG: Processing config 21 with 2 atoms
DEBUG: Processing config 22 with 2 atoms
DEBUG: Processing config 23 with 1 atoms
DEBUG: Processing config

In [29]:
fs.perform_fit()

'fit' took 1.20 ms on rank 0
'error_analysis' took 7.17 ms on rank 0


/Users/mitch/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/mitch/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/mitch/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/mitch/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=9.65546e-23): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
/Users/mitch/github/FitSNAP-alphataubio/examples/pacemaker/Cu-I/../../../fitsnap3lib/solvers/ridge.py:80: RuntimeWarning: divide by zero encountered in matmul
  residues = np.matmul(aw,reg.coef_) - bw
/Users/mitch/github/FitSNAP-alphata

In [26]:
print(fs.solver.errors)
print(fs.solver.df)

                                                            ncount       mae  \
Group                        Weighting  Testing  Subsystem                     
*ALL                         Unweighted Training Energy        105  0.004276   
                             weighted   Training Energy        105  0.004276   
Cu_df1_A1_A2_A3_EV_elast_pho Unweighted Training Energy        105  0.004276   
                             weighted   Training Energy        105  0.004276   

                                                                rmse       rsq  
Group                        Weighting  Testing  Subsystem                      
*ALL                         Unweighted Training Energy     0.005751  0.978667  
                             weighted   Training Energy     0.005751  0.978667  
Cu_df1_A1_A2_A3_EV_elast_pho Unweighted Training Energy     0.005751  0.978667  
                             weighted   Training Energy     0.005751  0.978667  
             0         1         